# Text Acquisition & Ingestion Assignment

In [ ]:
!pip install beautifulsoup4 --quiet
!pip install feedparser --quiet
import json
import requests
import feedparser
from bs4 import BeautifulSoup

     |████████████████████████████████| 194kB 3.4MB/s 


### Iterate through the list of article URLs below, scraping the text from each one and saving it to a text file. 

In [ ]:
articles = ['http://lite.cnn.io/en/article/h_eac18760a7a7f9a1bf33616f1c4a336d',
            'http://lite.cnn.io/en/article/h_de3f82f17d289680dd2b47c6413ebe7c',
            'http://lite.cnn.io/en/article/h_72f4dc9d6f35458a89af014b62e625ad',
            'http://lite.cnn.io/en/article/h_aa21fe6bf176071cb49e09d422c3adf0',
            'http://lite.cnn.io/en/article/h_8ad34a532921c9076cdc9d7390d2f1bc',
            'http://lite.cnn.io/en/article/h_84422c79110d9989177cfaf1c5f45fe7',
            'http://lite.cnn.io/en/article/h_d010d9580abac3a44c6181ec6fb63d58',
            'http://lite.cnn.io/en/article/h_fb11f4e9d7c5323e75b337d9e9e5e368',
            'http://lite.cnn.io/en/article/h_7b27f0b131067f8ece6238ac559670ab',
            'http://lite.cnn.io/en/article/h_8cae7f735fa9573d470f802063ceffe2',
            'http://lite.cnn.io/en/article/h_72c3668280e82576fcc2602b0fa70c14',
            'http://lite.cnn.io/en/article/h_d20658fb0e20212051cda0e0a7248c8a',
            'http://lite.cnn.io/en/article/h_56611c43d7928120d2ae21666ccc7417',
            'http://lite.cnn.io/en/article/h_bda0394e3c5ee7054ee65c022bca7695']

In [ ]:
def html_to_text(html):
    TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']

    soup = BeautifulSoup(html, "lxml")
    text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
    text = ' '.join(text_list)
    return text

In [ ]:
path = './news_articles/'

for index, url in enumerate(articles):
    response = requests.get(url)
    html = response.text
    text = html_to_text(html)
    
    with open(path + 'article_' + str(index) + '.txt', 'wb') as f:
        f.write(text.encode())

### Ingest the text files generated via web scraping into a corpus and print the corpus statistics.

In [ ]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

DOC_PATTERN = r'.*\.txt'
news_corpus = PlaintextCorpusReader('news_articles', DOC_PATTERN)

In [ ]:
def corpus_stats(corpus):
    print("Corpus Statistics")
    print("Number of documents: " + str(len(corpus.fileids())))
    print("Number of paragraphs: " + str(len(corpus.paras())))
    print("Number of sentences: " + str(len(corpus.sents())))
    print("Number of words: " + str(len(corpus.words())))
    print("Vocabulary: " + str(len(set(w.lower() for w in corpus.words()))))
    print("Avg chars per word: " + str(round(len(corpus.raw())/len(corpus.words()),1)))
    print("Avg words per sentence: " + str(round(len(corpus.words())/len(corpus.sents()),1)))

In [ ]:
corpus_stats(news_corpus)

### Parse the O'Reilly Radar RSS feed below, extract the text from each post, and save it to a text file.

The content of each post contains HTML tags. Strip those out using the same approach you used for web scraping so that only text is saved to the files.

In [ ]:
feed = 'http://feeds.feedburner.com/oreilly/radar/atom'

In [ ]:
parsed = feedparser.parse(feed)
posts = parsed.entries

path = './oreilly_rss/'

for index, post in enumerate(posts):
    content = posts[index]['content'][0]['value']
    text = html_to_text(content)
    with open(path + 'article_' + str(index) + '.txt', 'wb') as f:
        f.write(text.encode())

### Ingest the text files generated via RSS parsing into a corpus and print the corpus statistics.

In [ ]:
DOC_PATTERN = r'.*\.txt'
oreilly_corpus = PlaintextCorpusReader('oreilly_rss', DOC_PATTERN)
corpus_stats(oreilly_corpus)

### Make an API call to the Hacker News API to retrieve their Ask, Show, and Job category items. 

- URL: https://hacker-news.firebaseio.com/v0/askstories.json

In [ ]:
api = 'https://hacker-news.firebaseio.com/v0/askstories.json'
response = requests.get(api)
results = json.loads(response.content)

### Once you have retrieved the item IDs from the URL above, retrieve each item by adding the item ID to the URL below, extract the item's text property, and save the text from each item to disk as its own document.

- URL: https://hacker-news.firebaseio.com/v0/item/ITEM_ID_HERE.json

The content of some items may contain HTML tags. Strip those out using the same approach you used for web scraping so that only text is saved to the files.

In [ ]:
posts = []

for item in results:
    response = requests.get('https://hacker-news.firebaseio.com/v0/item/' + str(item) + '.json')
    try:
        post = json.loads(response.content)['text']
        posts.append(post)
    except:
        pass

In [ ]:
path = './hacker_news/'

for index, post in enumerate(posts):
    text = html_to_text(post)
    with open(path + 'article_' + str(index) + '.txt', 'wb') as f:
        f.write(text.encode())

### Ingest the text files generated via API into a corpus and print the corpus statistics.

In [ ]:
DOC_PATTERN = r'.*\.txt'
hn_corpus = PlaintextCorpusReader('hacker_news', DOC_PATTERN)
corpus_stats(hn_corpus)